In [1]:
import itertools
import os.path as op
import os

from gradec.decode import LDADecoder
from gradec.utils import _rm_medial_wall, _decoding_filter
from gradec.plot import plot_surf_maps, plot_radar, plot_cloud
from gradec.fetcher import _fetch_features, _fetch_frequencies, _fetch_classification
import nibabel as nib
import numpy as np

In [2]:

SPACE, DENSITY = "fsaverage", "164k"
DSET, MODEL = "neuroquery", "lda"

data_dir = op.join(".", "data")
neuromaps_dir = op.join(data_dir, "neuromaps")
figures_dir = op.join(data_dir, "figures")

# List of possible combinations of tracts, regions and smoothing
#tracts = tract_names
#regions = ["RAS", "LPI",'']
smths = ["", ".smooth_1"]
thresholds = ["0", "0.15", "0.25"]

In [3]:
decode = LDADecoder(space=SPACE, density=DENSITY, calc_pvals=False, data_dir=data_dir)
decode.fit(DSET)

# Load features for visualization
features = _fetch_features(DSET, MODEL, data_dir=data_dir)
frequencies = _fetch_frequencies(DSET, MODEL, data_dir=data_dir)
classification, class_lst = _fetch_classification(DSET, MODEL, data_dir=data_dir)

In [4]:
file_dir = os.getcwd() + "/data/white-matter-atlas_thresholds/cortexmap_binarize_smooth-surf-1_threshold-0.15_dilate-0/cortexmap/func"
file_names = []

# extract all .gii file names
for x in os.listdir(file_dir):
    #print(x)
    if x.endswith(".gii"):
        file_names.append(x)

### making two seperate lists, one containing two RIO file names and other containing one ROI file names
files_mono = []
files_bi = []

for i in file_names:
    #print(i)
    if i[3]=='l' or i[3]=='r':
        files_bi.append(i)
    else:
        files_mono.append(i)

### Separated Endpoints

In [ ]:

separated_results={}

mono_df = pd.DataFrame(columns=['feature', 0, 'endpoint']) ### creating empy dataframe to store each corrs_df

### splitting 2 endpoints vs 1 endpoint
regions_bi = [] # tracts with 2 endpoint ROIs
regions_mono = [] # tracts with 1 endpoint ROIs

for i in file_names:
    split_region = i.split("_")[0][3:]
    if split_region.find('left') == 0:
        regions_bi.append(split_region[4:])
    elif split_region.find('right') == 0:
        regions_bi.append(split_region[5:])
    else:
        regions_mono.append(split_region)

tracts_mono = regions_mono

### Working on mono files
for fig_i, (smth, file_name) in enumerate(itertools.product(smths, files_mono)):
    # Path to the maps
    
    #print(file_name)
    #print(threshold)
    #print(tract)
    regions_dir = op.join(
        data_dir, 
        "white-matter-atlas_thresholds", 
        f"cortexmap_binarize_smooth-surf-1_threshold-0_dilate-0", 
        "cortexmap", 
        "func",
    )

    
    ### handling smooth vs not
    if 'smooth' in file_name:
        smth = '.smooth_1'
    else:
        smth = ''


    for tract in tracts_mono:
        #print(tract)
        if file_name[0] == 'l':  ### These will be assigned as left
            if 'LPI' in file_name:
                map_lh = op.join(regions_dir, f'lh.{tract}_box_1mm_LPI_FiberEndpoint{smth}.func.gii')
                map_arr_lh = nib.load(map_lh).agg_data()
            else: 
                map_lh = op.join(regions_dir, f'lh.{tract}_box_1mm_RAS_FiberEndpoint{smth}.func.gii')
                map_arr_lh = nib.load(map_lh).agg_data()
            
        else:   ### these will be assigned right
            if 'LPI' in file_name:
                map_rh = op.join(regions_dir, f'rh.{tract}_box_1mm_LPI_FiberEndpoint{smth}.func.gii')
                map_arr_rh = nib.load(map_rh).agg_data()
            else: 
                map_rh = op.join(regions_dir, f'rh.{tract}_box_1mm_RAS_FiberEndpoint{smth}.func.gii')
                map_arr_rh = nib.load(map_rh).agg_data()
            
        
        
        # Remove medial wall
        map_arr = _rm_medial_wall(
            map_arr_lh,
            map_arr_rh,
            space=SPACE,
            density=DENSITY,
            neuromaps_dir=neuromaps_dir,
        )

        # Decode map
        corrs_df = decode.transform([map_arr], method="correlation")
        filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
            corrs_df,
            features,
            classification,
            freq_by_topic=frequencies,
            class_by_topic=class_lst,
        )

        flat_df = corrs_df.reset_index() ### getting accuarte column names
        flat_df['endpoint'] = file_name[:48]   ### adding endpoint name
        mono_df = pd.concat([mono_df, flat_df], axis = 0, ignore_index=True)  ### adding endpoint df to complete df

        # filtered_df.columns = ["r"]
        # separated_results[file_name] = filtered_df.sort_values(by="r", ascending=False)
        tracts_mono.pop(0)
        break

    if len(tracts_mono) == 0:
        break
        

In [138]:
mono_df['r'] = mono_df[0]
mono_df = mono_df.drop([0], axis = 1)
mono_df

,feature,endpoint,r
0,1_insula_anterior insula_disgust,lh.anterioFrontalCC_box_1mm_LPI_FiberEndpoint.fu,-0.090924
1,2_picture_valence_arousal,lh.anterioFrontalCC_box_1mm_LPI_FiberEndpoint.fu,0.004395
2,3_system_complex_simple,lh.anterioFrontalCC_box_1mm_LPI_FiberEndpoint.fu,-0.057251
3,4_mpfc_physical_status,lh.anterioFrontalCC_box_1mm_LPI_FiberEndpoint.fu,0.159972
4,5_motion_sts_psts,lh.anterioFrontalCC_box_1mm_LPI_FiberEndpoint.fu,-0.054214
...,...,...,...
3995,196_amygdala_amygdala response_analysis,rh.parietalCC_box_1mm_RAS_FiberEndpoint.smooth_1,-0.170385
3996,197_context_information_schema,rh.parietalCC_box_1mm_RAS_FiberEndpoint.smooth_1,0.090420
3997,198_cue_cues_cued,rh.parietalCC_box_1mm_RAS_FiberEndpoint.smooth_1,0.166218
3998,199_naming_generation_production,rh.parietalCC_box_1mm_RAS_FiberEndpoint.smooth_1,-0.171840


In [139]:
mono_df.to_csv('mono_df.csv')